<h1>IBM Applied Data Science Capstone Course by Coursera</h1>  
<h3>Week 3 Part 1 & 2</h3>   
<h4>Task 1 : Build a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name in Toronto.</h4>
<h4>Task 2 : To get the latitude and the longitude coordinates of each neighborhood.

<h3>Step 1: Importinng all required libraries for the 3 parts of code 

In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library
import urllib
import bs4
print("Libraries imported.")

Libraries imported.


<h3> Using Beautiful soup library to parse data from the wikipedia page

In [4]:
#url link
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

#Parsing data from the url link using Beautiful soup package
source = urllib.request.urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').read()
soup = bs4.BeautifulSoup(source,'html.parser')

#Identifying table and its rows and saving them 
table = soup.find('table')
table_rows = table.find_all('tr')

#Create empty list and save the data from the page into it
l = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        l.append(row)

<h3> Creating Dataframe to store the data from the list into columns and naming them

In [5]:
df = pd.DataFrame(l, columns=["PostalCode", "Borough", "Neighbourhood"])
df.head(10)

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,None
1,M2A,Not assigned,None
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,None
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,"Malvern, Rouge"


<h3> Removing rows with value of " Not assigned"

In [6]:
df = df[df.Borough != 'Not assigned']
df.head(10)

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


<h3> More than one neighbour can exist in same postal code

In [7]:
df = df.groupby(['PostalCode', 'Borough']).agg(', '.join)
df = df.reset_index()
df.head(10)

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


<h3> " If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough." - from question

In [8]:
df.loc[df['Neighbourhood']=='Not assigned', ['Neighbourhood']] = 'Queen\'s Park'
df.head(10)

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


<h3>Shape of the dataframe as asked

In [9]:
df.shape

(103, 3)

<h3> Part 2 : Getting Latitude and Longitude 

In [10]:
!conda install -c conda-forge geocoder --yes 
import geocoder


Solving environment: done

# All requested packages already installed.



<h3> Reading the geospatial coordinates csv file into coordinates data frame

In [11]:
coordinates = pd.read_csv("Geospatial_Coordinates.csv")
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


<h3> Renaming the column name "Postal Code" to "PostalCode" so merge is proper

In [12]:
coordinates.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
coordinates.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


<h3>Merging the data  frames to produce required lattitude and longitude values for neighbourhood

In [18]:
toronto_df_new = df.merge(coordinates, on="PostalCode", how="left")
toronto_df_new.head(10)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


<h3> Checking the dataframe from question. Please ignore another column of Neighborhood

In [17]:
column_names = ["PostalCode", "Borough", "Neighborhood", "Latitude", "Longitude"]
test_df = pd.DataFrame(columns=column_names)

test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_list:
    test_df = test_df.append(toronto_df_new[toronto_df_new["PostalCode"]==postcode], ignore_index=True)
    
test_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Neighbourhood
0,M5G,Downtown Toronto,NaN,43.657952,-79.387383,Central Bay Street
1,M2H,North York,NaN,43.803762,-79.363452,Hillcrest Village
2,M4B,East York,NaN,43.706397,-79.309937,"Parkview Hill, Woodbine Gardens"
3,M1J,Scarborough,NaN,43.744734,-79.239476,Scarborough Village
4,M4G,East York,NaN,43.709060,-79.363452,Leaside
5,M4M,East Toronto,NaN,43.659526,-79.340923,Studio District
6,M1R,Scarborough,NaN,43.750072,-79.295849,"Wexford, Maryvale"
7,M9V,Etobicoke,NaN,43.739416,-79.588437,"South Steeles, Silverstone, Humbergate, Jamest..."
8,M9L,North York,NaN,43.756303,-79.565963,Humber Summit
9,M5V,Downtown Toronto,NaN,43.628947,-79.394420,"CN Tower, King and Spadina, Railway Lands, Har..."
